In [4]:
import pandas as pd
import geopandas as gpd
from sklearn.metrics import (
    confusion_matrix, cohen_kappa_score,
    accuracy_score, precision_score, recall_score, f1_score
)


def save_confusion_matrix(gdf, field_true, field_pred, csv_path):
    """
    Berechnet Confusion Matrix + Metriken mit festem Label-Mapping.
    Null/NaN-Werte werden entfernt.
    """

    # --- Hardcoded Mapping ---
    mapping = {
        1: "Forest land",
        2: "Cropland",
        3: "Grassland",
        4: "Wetlands",
        5: "Settlements",
        6: "Other Land"
    }

    # --- NaN-Zeilen filtern ---
    mask = ~(gdf[field_true].isna() | gdf[field_pred].isna())
    df = gdf.loc[mask, [field_true, field_pred]].copy()

    # --- Numerische Labels konsistent (1.0 → 1) ---
    def normalize_label(val):
        if pd.isna(val):
            return None
        try:
            f = float(val)
            if f.is_integer():
                return int(f)
            return f
        except:
            return str(val).strip()

    df[field_true] = df[field_true].apply(normalize_label)
    df[field_pred] = df[field_pred].apply(normalize_label)

    # --- Map Labels über hardcoded Mapping ---
    df[field_true] = df[field_true].map(mapping)
    df[field_pred] = df[field_pred].map(mapping)

    # --- y_true / y_pred ---
    y_true = df[field_true]
    y_pred = df[field_pred]

    # --- Alle Labels aus Mapping, die tatsächlich vorkommen ---
    labels = [mapping[k] for k in sorted(mapping.keys()) if mapping[k] in y_true.values or mapping[k] in y_pred.values]

    # --- Confusion Matrix ---
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_df = pd.DataFrame(cm, index=labels, columns=labels)
    cm_df.index.name = "true"
    cm_df.columns.name = "pred"

    # --- Metriken ---
    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, average="weighted", zero_division=0),
        "recall": recall_score(y_true, y_pred, average="weighted", zero_division=0),
        "f1_score": f1_score(y_true, y_pred, average="weighted", zero_division=0),
        "kappa": cohen_kappa_score(y_true, y_pred)
    }
    metrics_df = pd.DataFrame.from_dict(metrics, orient="index", columns=["value"])

    # --- CSV speichern ---
    with open(csv_path, "w", encoding="utf-8") as f:
        f.write("Confusion Matrix\n")
        cm_df.to_csv(f)
        f.write("\nMetrics\n")
        metrics_df.to_csv(f)

    return {
        "confusion_matrix": cm_df,
        "metrics": metrics_df,
        "labels": labels
    }


In [ ]:
import geopandas as gpd

In [7]:


gdf=gpd.read_file(r"C:\Users\aebim\Documents\02_Ausbildung\Studium\05_Semester\5230_Geoniformatik_Raumanalyse\Projektarbeit\03_GitHub\data\analysis\av\AV_As_Center_Pixel_ZH2.gpkg")
field_true="IPCC_AS_Id"
field_pred="IPCC_AV_Id"
csv_path= r"C:\Users\aebim\Documents\02_Ausbildung\Studium\05_Semester\5230_Geoniformatik_Raumanalyse\Projektarbeit\03_GitHub\data\analysis\av\confusion_matrix1.csv"


result = save_confusion_matrix(gdf, field_true, field_pred, csv_path)

print(result["confusion_matrix"])
print(result["metrics"])


pred         Forest land  Cropland  Grassland  Wetlands  Settlements  \
true                                                                   
Forest land        48912      2086        246       366         1780   
Cropland              42     40117         22         1          520   
Grassland            196     29477        265         8          449   
Wetlands             150       406       1529      8605           46   
Settlements          530     15926        882        46        21358   
Other Land             5        42          5         3            3   

pred         Other Land  
true                     
Forest land          31  
Cropland             34  
Grassland            17  
Wetlands             12  
Settlements         417  
Other Land            1  
              value
accuracy   0.683290
precision  0.679463
recall     0.683290
f1_score   0.643102
kappa      0.581996
